In [1]:
class Counter:
    def __init__(self):
        self.count = 0

    def increment(self, offset):
        self.count += offset

In [2]:
def worker(sensor_index, how_many, counter):
    for _ in range(how_many):
        counter.increment(1)

In [4]:
from threading import Thread
how_many = 10**5
counter = Counter()
threads = []
for i in range(5):
    thread = Thread(target=worker, args=(i, how_many, counter))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

expected = how_many* 5
found = counter.count
print(f'Counter should be {expected}, got {found}')

Counter should be 500000, got 295713


In [5]:
#データ競合やデータ構造の破壊を防ぐために相互排他ロックのLock
from threading import Lock
class LockingCounter:
    def __init__(self):
        self.lock = Lock()
        self.count = 0

    def increment(self, offset):
        with self.lock:
            self.count += offset

In [6]:
counter = LockingCounter()
threads = []
for i in range(5):
    thread = Thread(target=worker,
                    args=(i, how_many, counter))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()


In [8]:
expected = how_many*5 
print(counter.count)

500000
